Modify this line to briefly discribe the functionality of new_script.ipynb<br/><br/>Copyright (C) 2017  Martin Engqvist Lab<br/>This program is free software: you can redistribute it and/or modify<br/>it under the terms of the GNU General Public License as published by<br/>the Free Software Foundation, either version 3 of the License, or<br/>(at your option) any later version.<br/>This program is distributed in the hope that it will be useful,<br/>but WITHOUT ANY WARRANTY; without even the implied warranty of<br/>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br/>GNU General Public License for more details.<br/>You should have received a copy of the GNU General Public License<br/>along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from os.path import join, dirname, basename, exists, isdir

### Load environmental variables from the project root directory ###
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

# now you can get the variables using their names

# Check whether a network drive has been specified
DATABASE = os.environ.get("NETWORK_URL")
if DATABASE == 'None':
    pass
else:
    pass
    #mount network drive here

# set up directory paths
CURRENT_DIR = os.getcwd()
PROJ = dirname(dotenv_path) # project root directory

DATA = join(PROJ, 'data') #data directory
RAW_EXTERNAL = join(DATA, 'raw_external') # external data raw directory
RAW_INTERNAL = join(DATA, 'raw_internal') # internal data raw directory
INTERMEDIATE = join(DATA, 'intermediate') # intermediate data directory
FINAL = join(DATA, 'final') # final data directory

RESULTS = join(PROJ, 'results') # output directory
FIGURES = join(RESULTS, 'figures') # figure output directory
PICTURES = join(RESULTS, 'pictures') # picture output directory


# make folders specific for certain data
folder_name = ''
if folder_name != '':
    #make folders if they don't exist
    if not exists(join(RAW_EXTERNAL, folder_name)):
        os.makedirs(join(RAW_EXTERNAL, folder_name))

    if not exists(join(INTERMEDIATE, folder_name)):
        os.makedirs(join(INTERMEDIATE, folder_name))

    if not exists(join(FINAL, folder_name)):
        os.makedirs(join(FINAL, folder_name))


print('Standard variables loaded, you are good to go!')

Standard variables loaded, you are good to go!


# Read in data
here, data is read in as well as translations from uniprot ID to NCBI Entrez (mapper obtained from: https://www.uniprot.org/uploadlists/)

In [2]:
## Your code here ##
import pandas as pd

# de results loading
#de_results = pd.read_csv("~/Documents/masters/thesis/thesis/data/final/de_edger_results_stress.csv")

# protein to gene translation file read in 
uniprot_to_entrez = pd.read_csv("~/Documents/masters/thesis/thesis/data/intermediate/uniprot_to_entrez.tsv", sep="\t")
uniprot_to_entrez = uniprot_to_entrez.drop_duplicates(subset ="From", keep = False)\
.set_index("From").to_dict('index')


FileNotFoundError: [Errno 2] File b'/Users/jonas/Documents/masters/thesis/thesis/data/intermediate/uniprot_to_entrez.tsv' does not exist: b'/Users/jonas/Documents/masters/thesis/thesis/data/intermediate/uniprot_to_entrez.tsv'

In [4]:
uniprot_to_entrez

NameError: name 'uniprot_to_entrez' is not defined

In [5]:
# de results translated to gene IDs
de_results_entrez = de_results
de_results_entrez["Unnamed: 0"] = [uniprot_to_entrez[i]['To'] if i in uniprot_to_entrez else "nan" for i in de_results_entrez["Unnamed: 0"]]
de_results_entrez = de_results_entrez[de_results_entrez["Unnamed: 0"] != "nan"]
de_results_ids = [list(de_results[de_results[i] != 0]["Unnamed: 0"]) for i in de_results.columns]


NameError: name 'de_results' is not defined

In [13]:
# Get ontologies (from http://geneontology.org/ontology/go-basic.obo)
from goatools.base import download_go_basic_obo
from goatools.obo_parser import GODag
import os
obo_fname = download_go_basic_obo()  # Download data
obodag = GODag(obo_fname)  # Load data

# Get associations genes <-> ontologies (from ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz)
from goatools.base import download_ncbi_associations
from goatools.anno.genetogo_reader import Gene2GoReader
gene2go_fname = download_ncbi_associations()  # Download data
objanno = Gene2GoReader(gene2go_fname, taxids=[511145])  # Read data (only for e. coli)
ns2assoc = objanno.get_ns2assc()

#Remove downloaded data:
os.remove(obo_fname)
os.remove(gene2go_fname)

requests.get(http://purl.obolibrary.org/obo/go/go-basic.obo, stream=True)
  WROTE: go-basic.obo

go-basic.obo: fmt(1.2) rel(2020-02-21) 47,245 GO Terms
FTP RETR ftp.ncbi.nlm.nih.gov gene/DATA gene2go.gz -> gene2go.gz
  gunzip gene2go.gz
HMS:0:00:03.399185  43,485 annotations,  3,893 genes,  4,109 GOs, 1 taxids READ: gene2go 


# Read in translation from gene to go 

chosen taxid E. coli corresponds to Escherichia coli str. K-12 substr. MG1655, as found here: https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&id=511145&lvl=3&lin=f&keep=1&srchmode=1&unlock 


In [6]:
from __future__ import print_function
from goatools.anno.genetogo_reader import Gene2GoReader

for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated e. coli genes".format(NS=nspc, N=len(id2gos)))

NameError: name 'ns2assoc' is not defined

In [1]:
# Over-representation object:
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
population = de_results_entrez["Unnamed: 0"]

goeaobj = GOEnrichmentStudyNS(
    population,  # Population (all mapped yeast genes in Entrez format)
    ns2assoc,  # geneid/GO associations
    obodag,  # Ontologies
    propagate_counts = False,  # ???
    alpha = 0.05,  # default significance cut-off
    methods = ['fdr_bh'])  # default multipletest correction method

NameError: name 'de_results_entrez' is not defined

# Run Gene ontology
run gene ontology enrichment on the top genes contained within the 

In [29]:
%%capture
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
# genes to be analysed
geneids_study = list(de_results_entrez["Unnamed: 0"][0:400])

goea_results_sig = []
for i in de_results_ids:
    goea_results_all = goeaobj.run_study(i)
    goea_results_sig.append([r for r in goea_results_all if r.p_fdr_bh < 0.05])

 88%  1,764 of  1,997 study items found in association
 97%  1,997 of  2,057 study items found in population(1997)
Calculating 1,470 uncorrected p-values using fisher_scipy_stats
   1,470 GO terms are associated with  1,764 of  1,997 population items
   1,470 GO terms are associated with  1,764 of  1,998 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
 81%  1,619 of  1,997 study items found in association
 97%  1,997 of  2,057 study items found in population(1997)
Calculating 167 uncorrected p-values using fisher_scipy_stats
     167 GO terms are associated with  1,619 of  1,997 population items
     167 GO terms are associated with  1,619 of  1,998 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr

   1,470 GO terms are associated with  1,764 of  1,997 population items
   1,240 GO terms are associated with  1,237 of  1,396 study items
  METHOD fdr_bh:
       1 GO terms found significant (< 0.05=alpha) (  1 enriched +   0 purified): statsmodels fdr_bh
      75 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
 84%  1,173 of  1,395 study items found in association
 96%  1,395 of  1,447 study items found in population(1997)
Calculating 167 uncorrected p-values using fisher_scipy_stats
     167 GO terms are associated with  1,619 of  1,997 population items
     141 GO terms are associated with  1,173 of  1,396 study items
  METHOD fdr_bh:
       6 GO terms found significant (< 0.05=alpha) (  3 enriched +   3 purified): statsmodels fdr_bh
     815 study items associated with significant GO IDs (enriched)
     302 study items associated with significant GO IDs (purified)
 90%  1,251 of  1,395 study items found i

   1,470 GO terms are associated with  1,764 of  1,997 population items
   1,346 GO terms are associated with  1,430 of  1,611 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
 83%  1,336 of  1,610 study items found in association
 97%  1,610 of  1,660 study items found in population(1997)
Calculating 167 uncorrected p-values using fisher_scipy_stats
     167 GO terms are associated with  1,619 of  1,997 population items
     153 GO terms are associated with  1,336 of  1,611 study items
  METHOD fdr_bh:
       2 GO terms found significant (< 0.05=alpha) (  2 enriched +   0 purified): statsmodels fdr_bh
     734 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
 90%  1,443 of  1,610 study items found i

   1,470 GO terms are associated with  1,764 of  1,997 population items
   1,247 GO terms are associated with  1,300 of  1,475 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
 83%  1,227 of  1,474 study items found in association
 97%  1,474 of  1,525 study items found in population(1997)
Calculating 167 uncorrected p-values using fisher_scipy_stats
     167 GO terms are associated with  1,619 of  1,997 population items
     141 GO terms are associated with  1,227 of  1,475 study items
  METHOD fdr_bh:
       3 GO terms found significant (< 0.05=alpha) (  3 enriched +   0 purified): statsmodels fdr_bh
     835 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
 89%  1,309 of  1,474 study items found i

   1,470 GO terms are associated with  1,764 of  1,997 population items
   1,275 GO terms are associated with  1,285 of  1,448 study items
  METHOD fdr_bh:
       2 GO terms found significant (< 0.05=alpha) (  1 enriched +   1 purified): statsmodels fdr_bh
      82 study items associated with significant GO IDs (enriched)
      21 study items associated with significant GO IDs (purified)
 84%  1,218 of  1,447 study items found in association
 96%  1,447 of  1,502 study items found in population(1997)
Calculating 167 uncorrected p-values using fisher_scipy_stats
     167 GO terms are associated with  1,619 of  1,997 population items
     144 GO terms are associated with  1,218 of  1,448 study items
  METHOD fdr_bh:
       4 GO terms found significant (< 0.05=alpha) (  2 enriched +   2 purified): statsmodels fdr_bh
     680 study items associated with significant GO IDs (enriched)
     293 study items associated with significant GO IDs (purified)
 90%  1,305 of  1,447 study items found i

In [31]:
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj

# goeaobj.wr_tsv(join(FINAL, "p_values_" + group_name + ".tsv"), goea_results_sig)

# save results of goe analysis
j = 0
for i in de_results.columns[1:]:
    goeaobj.wr_tsv("/Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_{}.tsv".format(i), goea_results_sig[j])
    j += 1

      0 items. NOT WRITING /Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_LB-Glucose.tsv
      9 items WROTE: /Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_Glycerol...AA-Glucose.tsv
     12 items WROTE: /Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_Acetate-Glucose.tsv
     10 items WROTE: /Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_Fumarate-Glucose.tsv
     10 items WROTE: /Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_Glucosamine-Glucose.tsv
      7 items WROTE: /Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_Glycerol-Glucose.tsv
      9 items WROTE: /Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_Pyruvate-Glucose.tsv
      6 items WROTE: /Users/jonas/Documents/masters/thesis/thesis/data/final/go_analysis/p_values_go_Chemostat.µ.0.5-Glucose.tsv
      2 it

In [94]:
# plot_results("/Users/jonas/Documents/masters/thesis/thesis/results/figures/nbt3102_{NS}.png", goea_results_sig)

FileNotFoundError: [Errno 2] "dot" not found in path.